In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gensim
import gensim.models.keyedvectors as word2vec
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from pandarallel import pandarallel

In [3]:
from parse_data import parseData

In [4]:
pandarallel.initialize(progress_bar=True,nb_workers=15,shm_size_mb=2500)


New pandarallel memory created - Size: 2500 MB
Pandarallel will run on 15 workers


In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
baseline_f = pd.read_csv('./www2018-table/feature/features.csv')

In [7]:
w2v_gn_model = word2vec.KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin", binary=True)  

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# w2v_gn_model.similarity('computer','ram')

In [ ]:
# w2v_gn_model.get_vector('computer')

In [8]:
rdv2vec_model = gensim.models.Word2Vec.load('./DB2Vec_sg_200_5_5_15_2_500')

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# rdv2vec_model.wv.similarity('dbr:England','dbr:United_Kingdom')

In [ ]:
rdv2vec_model.wv['dbr:SNCF_TGV_Atlantique']

In [9]:
tables_path = '/home/vibhav/table2vec/tables_redi2_1/'

def get_table_path(table_id):
    return tables_path + 're_tables-' + table_id.split('-')[1] + '.json'

In [10]:
baseline_f['table_path'] = baseline_f.table_id.parallel_apply(get_table_path)

/home/vibhav/table2vec/tables_redi2_1/re_tables-1579.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1578.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1411.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1410.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1211.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1210.json















  

In [11]:
baseline_f['parsedTable'] = baseline_f.parallel_apply(lambda x: parseData(x['table_path'],x['table_id']).split(' '),axis=1)

In [17]:
def get_w2v_embd(sentence):
    embd = []
    for word in sentence:
        try:
            embd.append(w2v_gn_model.get_vector(word))
        except Exception as e:
            embd.append(w2v_gn_model.get_vector('UNK'))
    embd = np.array(embd)
    return embd

In [13]:
baseline_f['w2v_embd_table'] = baseline_f.parsedTable.parallel_apply(get_w2v_embd)

In [14]:
baseline_f.head()

,query_id,query,table_id,row,col,nul,in_link,out_link,pgcount,tImp,tPF,leftColhits,SecColhits,bodyhits,PMI,qInPgTitle,qInTableTitle,yRank,csr_score,idf1,idf2,idf3,idf4,idf5,idf6,max,sum,avg,sim,emax,esum,eavg,esim,cmax,csum,cavg,csim,remax,resum,reavg,resim,query_l,rel,table_path,parsedTable,w2v_embd_table
0,1,world interest rates Table,table-0875-680,8,2,0,31,21,51438,1.0,0.000260,0,0,0,0.000000,0.333333,0.222222,100,7.467415e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,9.266177,0.110312,0.676819,0.952804,89.707105,0.815519,0.971854,0.666667,5.291894,0.048108,0.354686,0.241209,3.716354,0.033785,0.281130,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[experian, score, expected, annual, loss, rate...","[[-0.09472656, -0.51171875, 0.05810547, 0.5078..."
1,1,world interest rates Table,table-1020-619,4,3,0,18,0,324,1.0,0.000786,0,0,0,-0.231049,0.000000,0.285714,100,8.663725e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,3.038269,0.108510,0.863125,1.000000,83.375200,0.757956,0.983893,1.000000,11.116121,0.101056,0.718895,1.000000,8.075247,0.073411,0.710250,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[annual, inflation, rates, aug, sep, silver, u...","[[0.0390625, 0.01586914, -0.07373047, 0.185546..."
2,1,world interest rates Table,table-0288-531,3,5,0,23,22,26419,0.5,0.000158,0,0,0,-0.069315,0.000000,0.000000,100,2.142316e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,0.234136,1.747577,0.043689,0.206247,0.841653,57.442424,0.638249,0.753198,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[regret, interest, rates, rise, static, rates,...","[[0.01928711, 0.07519531, 0.34179688, 0.020874..."
3,1,world interest rates Table,table-0288-530,4,5,1,23,22,26419,0.5,0.000211,0,0,0,-0.069315,0.000000,0.000000,100,1.888429e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,0.234136,1.641433,0.041036,0.252283,0.841653,57.442424,0.638249,0.753198,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[return, interest, rates, rise, static, rates,...","[[0.05126953, 0.12597656, -0.13769531, 0.08837..."
4,1,world interest rates Table,table-1000-57,2,2,0,38,1,2268,1.0,0.000130,0,0,0,0.000000,0.000000,0.500000,100,7.673589e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,4.982802,0.113245,0.700773,0.932809,89.552347,0.814112,0.954395,1.000000,10.147388,0.092249,0.372667,0.226134,4.564622,0.041497,0.279899,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[security, nominal, interest, rates, chakravar...","[[0.018920898, 0.18554688, 0.043945312, -0.070..."


In [15]:
semantic_f = baseline_f.loc[:,['query_id','query','table_path','table_id','parsedTable','rel']]
baseline_f.drop(columns=['table_path','parsedTable'],inplace=True)

In [18]:
semantic_f['w2v_embd_query'] = semantic_f['query'].parallel_apply(lambda x: get_w2v_embd(x.split(" ")))

NameError: name 'get_w2v_embd' is not defined

In [ ]:
def early_fusion(table,query):
    a = np.average(table,axis=0).reshape(1,-1)
    b = np.average(query,axis=0).reshape(1,-1)
    sim = cosine_similarity(a,b)
    return sim.reshape(-1)[0]

In [ ]:
semantic_f['w2v_early_fusion'] = semantic_f.apply(lambda x: early_fusion(x['w2v_embd_table'],x['w2v_embd_query']),axis=1)

In [ ]:
# semantic_f[semantic_f.w2v_early_fusion < 0.3].rel.value_counts()
# semantic_f[semantic_f.w2v_late_fusion_avg < 0.3].shape

In [ ]:
def late_fusion(table,query):
    s = []
    for i in query:
        for j in table:
            sim = cosine_similarity(i.reshape(1,-1),j.reshape(1,-1))
            s.append(sim)
    s = np.array(s).reshape(-1)
    return s

In [ ]:
semantic_f['w2v_late_fusion'] = semantic_f.parallel_apply(lambda x: late_fusion(x['w2v_embd_table'],x['w2v_embd_query']),axis=1)

In [ ]:
semantic_f['w2v_late_fusion_max'] = semantic_f.w2v_late_fusion.parallel_apply(np.max)
semantic_f['w2v_late_fusion_avg'] = semantic_f.w2v_late_fusion.parallel_apply(np.average)
semantic_f['w2v_late_fusion_sum'] = semantic_f.w2v_late_fusion.parallel_apply(np.sum)

In [ ]:
semantic_f.head()

In [19]:
from get_entities import getEntities

In [20]:
query_entities = {}
for i in semantic_f['query'].unique():
    query_entities[i] = getEntities(i)

semantic_f['query_entites'] = semantic_f['query'].apply(lambda x: query_entities[x])

Before: world interest rates Table
After: world interest rates Table
python -m nordlys.services.er -c config.json -q "world interest rates Table" CMD:RESULT b''


SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [ ]:
semantic_f.query_entites.iloc[1]

In [21]:
df_1 = pd.read_csv('./semantic_f_w2v_entities_1.csv')
df_2 = pd.read_csv('./semantic_f_w2v_entities_2.csv')
df_3 = pd.read_csv('./semantic_f_w2v_entities_3.csv')
df_4 = pd.read_csv('./semantic_f_w2v_entities_4.csv')

FileNotFoundError: File b'./semantic_f_w2v_entities_1.csv' does not exist

In [ ]:
df = pd.concat([df_1,df_2,df_3,df_4])

In [ ]:
import ast
df['table_entities'] = df.table_entities.apply(lambda x: ast.literal_eval(x))

In [ ]:
def get_g2v_embd(entities):
    embd = []
    for en in entities:
        word = 'dbr:' + en
        try:
            embd.append(rdv2vec_model.wv[word])
        except Exception as e:
            embd.append(rdv2vec_model.wv['dbr:UNK'])
    return np.array(embd)

In [ ]:
df['g2v_embd_table'] = df.table_entities.apply(get_g2v_embd)

In [ ]:
df['g2v_embd_query'] = df.query_entites.apply(get_g2v_embd)

In [ ]:
df['g2v_early_fusion'] = df.apply(lambda x: early_fusion(x['g2v_embd_table'],x['g2v_embd_query']),axis=1)

In [ ]:
df['g2v_late_fusion'] = df.parallel_apply(lambda x: late_fusion(x['g2v_embd_table'],x['g2v_embd_query']),axis=1)

In [ ]:
df['g2v_late_fusion_max'] = df.g2v_late_fusion.apply(np.max)
df['g2v_late_fusion_avg'] = df.g2v_late_fusion.apply(np.average)
df['g2v_late_fusion_sum'] = df.g2v_late_fusion.apply(np.sum)

In [ ]:
df.head()

In [ ]:
df.to_csv('./semantic_f_w2v_g2v.csv',index=False)